In [0]:
print("hello world")

hello world


In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import col,isnan,when,count,length,countDistinct,desc,array,size,length,date_format,to_date
from pyspark.sql.functions import broadcast,lit,struct,sum,round,datediff,year,month,concat,avg,max,min,when
import pyspark, unittest
from pyspark.sql import SparkSession
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Functions

In [0]:
#Reading a cvd file into RDD
def read_data_to_rdd(path):
    try:
        df = spark.read.format("csv").option("header","true").schema(schema).load(path).dropna(how='all')
        return df
    except Exception as e:
        print(e)

#Create a temp view for an RDD file        
def create_temp_view(df,temp_view_name):
    try:
        res = df.createOrReplaceTempView(temp_view_name) 
        return res
    except Exception as e:
        print(e)
        

#Checking null values function
def checking_null_values(df):
    return df.select([count(when(col(clm).isNull(), clm)).alias(clm) for clm in df.columns])
    
#Checking count of the rows and the distinct count of the primary key    
def check_column_count_distinct(df,pk):
    return df.select(count("*"), countDistinct(pk))
    
#Grouping a column to find the number of occurences    
def grpby_column_repeated_values(df,*pk,alias="count"):
    return df.groupBy(*pk).agg(count("*").alias(alias)).sort(desc(alias))

  
#Returnng the rows with the credit score greater than the given value
def compare_credit_score(df,clm,credit):
    res = df.select("CreditScore","Exited",clm).filter(df.CreditScore>credit)
    return res
    
    
#Returnng the rows with the balance equal to the given value
def comapre_zero_balance(df,clm,bal):
    res = df.select("Balance","Exited",clm).filter(df.Balance==bal)
    return res
    
    
#Returnng the rows with the tenure greater than the given value
def compare_tenure(df,clm,tenure):
    res = df_bank.select("Tenure","Exited",clm).filter(df_bank.Tenure>tenure)
    return res
    
    
#Returns the rows with products enrolled greater than the given value    
def no_of_prod(df,num):
    res = df.select("*").filter(df.NumOfProducts>num)
    return res


#Outpus the dataframe showing highest and lowest values in integer columns
def highest_lowest(df,clm,value):
    res = df.select("CustomerId","Surname",clm).sort([desc(clm) if value=="max" else clm])
    return res


#Groups a particular column by using avg function
def grpby_avg(df,avg_clm,*pk,alias="Avg"):
    return df.groupBy(*pk).agg(avg(avg_clm).alias(alias)).sort(desc(alias))



#Groups a particular column by using sum function
def grpby_sum(df,sum_clm,*pk,alias="Sum"):
    return df.groupBy(*pk).agg(sum(sum_clm).alias(alias)).sort(desc(alias))


    

Reading CSV file

In [0]:
schema = StructType([StructField("RowNumber",StringType(),True),StructField("CustomerId",StringType(),True),StructField("Surname",StringType(),True),StructField("CreditScore",IntegerType(),True),StructField("Geography",StringType(),True),StructField("Gender",StringType(),True),StructField("Age",IntegerType(),True),StructField("Tenure",IntegerType(),True),StructField("Balance",FloatType(),True),StructField("NumOfProducts",IntegerType(),True),StructField("HasCrCard",IntegerType(),True),StructField("IsActiveMember",IntegerType(),True),StructField("EstimatedSalary",FloatType(),True),StructField("Exited",IntegerType(),True)])



file_loc = "/FileStore/shared_uploads/manim31899@gmail.com"

df_bk = read_data_to_rdd(file_loc + "/Bank_credit_card_details.csv/")

In [0]:
df_bank = df_bk.withColumn(
    'AgeGroup',
     when((col("Age") >=10) & (col("Age") <20), "10s(Teens)")\
    .when((col("Age") >=20) & (col('Age') <30) , "20s(Twenties)")\
    .when((col("Age") >=30) & (col('Age') <40) , "30s(Thirties)")\
    .when((col("Age") >=40) & (col('Age') <50) , "40s(Fourties)")\
    .when((col("Age") >=50) & (col('Age') <60) , "50s(Fifties)")\
    .when((col("Age") >=60) & (col('Age') <70) , "60s(Sixties)")\
    .when((col("Age") >=70) & (col('Age') <80) , "70s(Seventies)")\
    .when((col("Age") >=80) & (col('Age') <90) , "80s(Eighties)")\
    .when((col("Age") >=90) & (col('Age') <100) , "90s(Nineties)")\
    .otherwise("Unknown")
)

df_bank

In [0]:
display(df_bank)

RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,AgeGroup
1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,101348.88,1,40s(Fourties)
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,40s(Fourties)
3,15619304,Onio,502,France,Female,42,8,159660.8,3,1,0,113931.57,1,40s(Fourties)
4,15701354,Boni,699,France,Female,39,1,0.0,2,0,0,93826.63,0,30s(Thirties)
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.1,0,40s(Fourties)
6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.7,1,40s(Fourties)
7,15592531,Bartlett,822,France,Male,50,7,0.0,2,1,1,10062.8,0,50s(Fifties)
8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1,20s(Twenties)
9,15792365,He,501,France,Male,44,4,142051.06,2,0,1,74940.5,0,40s(Fourties)
10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0,20s(Twenties)


Using Spark SQL to print the Customers who have not Exited the Credit Card Plan

In [0]:
df_bank_view = create_temp_view(df_bank,"df_bank_view")
display(spark.sql("SELECT * FROM df_bank_view WHERE Exited==0"))

RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,AgeGroup
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,40s(Fourties)
4,15701354,Boni,699,France,Female,39,1,0.0,2,0,0,93826.63,0,30s(Thirties)
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.1,0,40s(Fourties)
7,15592531,Bartlett,822,France,Male,50,7,0.0,2,1,1,10062.8,0,50s(Fifties)
9,15792365,He,501,France,Male,44,4,142051.06,2,0,1,74940.5,0,40s(Fourties)
10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0,20s(Twenties)
11,15767821,Bearce,528,France,Male,31,6,102016.72,2,0,0,80181.12,0,30s(Thirties)
12,15737173,Andrews,497,Spain,Male,24,3,0.0,2,1,0,76390.01,0,20s(Twenties)
13,15632264,Kay,476,France,Female,34,10,0.0,2,1,0,26260.98,0,30s(Thirties)
14,15691483,Chin,549,France,Female,25,5,0.0,2,0,0,190857.8,0,20s(Twenties)


Checking for any null values and whether each CustomerId is unique

In [0]:

display(checking_null_values(df_bank))
display(grpby_column_repeated_values(df_bank,"CustomerId"))

RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,AgeGroup
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


CustomerId,count
15693683,1
15724282,1
15588350,1
15637979,1
15775306,1
15794048,1
15743532,1
15596493,1
15751158,1
15706163,1


Customer Split based on Gender and Country

In [0]:
df_gen= grpby_column_repeated_values(df_bank,"Gender",alias="No_of_customers")
display(df_gen)

df_geo = grpby_column_repeated_values(df_bank,"Geography",alias="No_of_customers")
display(df_geo)

Gender,No_of_customers
Male,5457
Female,4543


Geography,No_of_customers
France,5014
Germany,2509
Spain,2477


Customer Surnames are grouped and found out the count

In [0]:
df_name = grpby_column_repeated_values(df_bank,"Surname",alias="No_of_customers")
display(df_name)

Surname,No_of_customers
Smith,32
Scott,29
Martin,29
Walker,28
Brown,26
Shih,25
Genovese,25
Yeh,25
Wright,24
Maclean,24


Number of Customers that are Active and Exited

In [0]:
df_exit = grpby_column_repeated_values(df_bank,"Exited",alias="No_of_customers")
display(df_exit)

df_act_mem = grpby_column_repeated_values(df_bank,"IsActiveMember",alias="No_of_customers")
display(df_act_mem)

Exited,No_of_customers
0,7963
1,2037


IsActiveMember,No_of_customers
1,5151
0,4849


Customers Grouped with Gender and Geography based on the Credit Score

In [0]:

df_gen_cred_temp = compare_credit_score(df_bank,"Gender",600)
df_gen_cred = grpby_column_repeated_values(df_gen_cred_temp,"Gender",alias="No_of_customers[CreditScore>600]")
display(df_gen_cred)

df_geo_cred_temp = compare_credit_score(df_bank,"Geography",600)
df_geo_cred = grpby_column_repeated_values(df_geo_cred_temp,"Geography",alias="No_of_customers[CreditScore>600]")
display(df_geo_cred)

Gender,No_of_customers[CreditScore>600]
Male,3773
Female,3161


Geography,No_of_customers[CreditScore>600]
France,3463
Germany,1741
Spain,1730


Customers from their Geography and Genders having Balance as Zero

In [0]:

df_geo_bal_temp = comapre_zero_balance(df_bank,"Geography",0)
df_geo_bal = grpby_column_repeated_values(df_geo_bal_temp,"Geography",alias= "No_of_customers[Balance=0]")
display(df_geo_bal)
    
    
    
df_bal_temp = comapre_zero_balance(df_bank,"Gender",0)
df_gen_bal = grpby_column_repeated_values(df_bal_temp,"Gender",alias= "No_of_customers[Balance=0]")
display(df_gen_bal)


Geography,No_of_customers[Balance=0]
France,2418
Spain,1199


Gender,No_of_customers[Balance=0]
Male,1963
Female,1654


Number of Customers from Countries that have more than 5 years of Tenure

In [0]:

df_geo_tenure_temp = compare_tenure(df_bank,"Geography",5)
df_geo_tenure = grpby_column_repeated_values(df_geo_tenure_temp,"Geography",alias= "No_of_customers[Tenure>5]")
display(df_geo_tenure)


Geography,No_of_customers[Tenure>5]
France,2267
Spain,1114
Germany,1113


Details of customers who have enrolled in more than two bank products

In [0]:

df_num_prod = no_of_prod(df_bank,2)
display(df_num_prod)
display(check_column_count_distinct(df_num_prod,"CustomerId"))

RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,AgeGroup
3,15619304,Onio,502,France,Female,42,8,159660.8,3,1,0,113931.57,1,40s(Fourties)
8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1,20s(Twenties)
31,15589475,Azikiwe,591,Spain,Female,39,3,0.0,3,1,0,140469.38,1,30s(Thirties)
71,15703793,Konovalova,738,Germany,Male,58,2,133745.44,4,1,0,28373.86,1,50s(Fifties)
89,15622897,Sharpe,646,France,Female,46,4,0.0,3,1,0,93251.42,1,40s(Fourties)
91,15757535,Heap,647,Spain,Female,44,5,0.0,3,1,1,174205.22,1,40s(Fourties)
228,15676895,Cattaneo,547,Germany,Female,39,6,74596.15,3,1,1,85746.52,1,30s(Thirties)
238,15764866,Synnot,539,Germany,Female,43,3,116220.5,3,1,0,55803.96,1,40s(Fourties)
239,15794056,Johnston,668,France,Female,46,2,0.0,3,1,0,89048.46,1,40s(Fourties)
272,15619955,Bevington,733,Germany,Male,34,3,100337.96,3,1,0,48559.19,1,30s(Thirties)


count(1),count(DISTINCT CustomerId)
326,326


Customers with highest and lowest credit score

In [0]:
    
df_high_cred = highest_lowest(df_bank,df_bank.CreditScore,"max")
display(df_high_cred)

df_low_cred = highest_lowest(df_bank,df_bank.CreditScore,"min")
display(df_low_cred)

CustomerId,Surname,CreditScore
15604482,Chiemezie,850
15733247,Stevenson,850
15607178,Welch,850
15602500,Maslova,850
15782468,Hart,850
15768219,Sung,850
15663883,Hansen,850
15602497,Honore,850
15649124,Fang,850
15747014,Pisani,850


CustomerId,Surname,CreditScore
15765173,Lin,350
15758813,Campbell,350
15668309,Maslow,350
15803202,Onyekachi,350
15685372,Azubuike,350
15679249,Chou,351
15692416,Aikenhead,358
15612494,Panicucci,359
15779947,Thomas,363
15597896,Ozoemena,365


Customers having the maximum and minimum balance

In [0]:
df_high_bal = highest_lowest(df_bank,df_bank.Balance,"max")
display(df_high_bal)

df_low_bal = highest_lowest(df_bank,df_bank.Balance,"min")
display(df_low_bal)

CustomerId,Surname,Balance
15757408,Lo,250898.1
15715622,To Rot,238387.56
15714241,Haddon,222267.62
15571958,McIntosh,221532.8
15586674,Shaw,216109.88
15599131,Dilke,214346.95
15594408,Chia,213146.2
15769818,Moore,212778.2
15620268,Thomson,212696.31
15780212,Mao,212692.97


CustomerId,Surname,Balance
15745543,Hughes,0.0
15808760,Evseev,0.0
15726814,Walton,0.0
15657214,Hsia,0.0
15701354,Boni,0.0
15592531,Bartlett,0.0
15592089,Larsen,0.0
15632264,Kay,0.0
15602929,Wilson,0.0
15600882,Scott,0.0


Estimated High Income Customers vs Low Income Customers

In [0]:
df_high_sal = highest_lowest(df_bank,df_bank.EstimatedSalary,"max")
display(df_high_sal)

df_low_sal = highest_lowest(df_bank,df_bank.EstimatedSalary,"min")
display(df_low_sal)

CustomerId,Surname,EstimatedSalary
15662021,Lucciano,199992.48
15634359,Dyer,199970.73
15697270,Gannon,199953.33
15762331,Moss,199929.17
15709136,Adams,199909.31
15741719,DeRose,199862.75
15682834,Johnstone,199857.47
15804211,Oluchukwu,199841.31
15815656,Hopkins,199808.1
15687913,Mai,199805.62


CustomerId,Surname,EstimatedSalary
15791053,Lucciano,11.58
15679693,Walker,90.07
15786463,Hsing,91.75
15639662,Phillips,96.27
15602851,Ozioma,106.67
15623369,Clifton,123.07
15648735,Cashin,142.81
15718667,T'ien,143.34
15640421,Conway,178.19
15684198,McDonald,216.27


Customers that bought most and least products from the bank

In [0]:
df_high_num_prod = highest_lowest(df_bank,df_bank.NumOfProducts,"max")
display(df_high_num_prod)

df_low_num_prod = highest_lowest(df_bank,df_bank.NumOfProducts,"min")
display(df_low_num_prod)

CustomerId,Surname,NumOfProducts
15664754,Steele,4
15719508,Davis,4
15603851,Galkin,4
15602735,Kuo,4
15601594,Ifeanacho,4
15703793,Konovalova,4
15670374,Wright,4
15608528,Munro,4
15671591,Castiglione,4
15668283,Gardiner,4


CustomerId,Surname,NumOfProducts
15787772,Hansen,1
15729970,Ugochukwu,1
15573086,Millar,1
15647311,Hill,1
15569050,Farrell,1
15592389,H?,1
15750765,Sanders,1
15661507,Muldrow,1
15799811,Herrera,1
15625047,Yen,1


Customers who has the highest and lowest tenure in the bank

In [0]:
df_high_tenure = highest_lowest(df_bank,df_bank.Tenure,"max")
display(df_high_tenure)

df_low_tenure = highest_lowest(df_bank,df_bank.Tenure,"min")
display(df_low_tenure)

CustomerId,Surname,Tenure
15699461,Fiorentini,10
15760085,Calabresi,10
15698932,Groves,10
15651022,Yost,10
15656176,Jenkins,10
15733247,Stevenson,10
15797219,Ifesinachi,10
15572360,Clark,10
15800434,Burgess,10
15767339,Chiazagomekpere,10


CustomerId,Surname,Tenure
15645248,Ho,0
15708710,Ford,0
15627000,Freeman,0
15793726,Matveyeva,0
15647091,Endrizzi,0
15713826,Ferguson,0
15788483,Kerr,0
15739438,Reed,0
15812518,Palermo,0
15643824,Johnston,0


Comapring theTotal Customers with the Remaining Customers that HasCrCard, IsActiveMember, Exited

In [0]:
def count_num(df,clm):
    res = df.select(count("*").alias("Total Customers"),sum(clm).alias(clm))
    return res

display(count_num(df_bank,"HasCrCard"))
display(count_num(df_bank,"IsActiveMember"))
display(count_num(df_bank,"Exited"))


Total Customers,HasCrCard
10000,7055


Total Customers,IsActiveMember
10000,5151


Total Customers,Exited
10000,2037


Active Customers grouped by their Countries

In [0]:
df_act = df_bank.select("*").filter(df_bank.IsActiveMember==1)

df_act_country = grpby_column_repeated_values(df_act,"Geography",alias="No_of_active_customers")
display(df_act_country)

Geography,No_of_active_customers
France,2591
Spain,1312
Germany,1248


Gender Partition among Customers whi have not exited the Credit

In [0]:
df_not_exit = df_bank.select("*").filter(df_bank.Exited==0)

df_not_exit_country = grpby_column_repeated_values(df_not_exit,"Gender",alias="No_of_customers_not_exited")
display(df_not_exit_country)

Gender,No_of_customers_not_exited
Male,4559
Female,3404


Grouping Age Groups to Avg Credit Score

In [0]:

df_agegrp_avg_cred = grpby_avg(df_bank,"CreditScore","AgeGroup",alias="Avg_Cred_Score")
display(df_agegrp_avg_cred)

AgeGroup,Avg_Cred_Score
90s(Nineties),729.0
80s(Eighties),677.0
10s(Teens),675.0816326530612
70s(Seventies),665.4926470588235
30s(Thirties),651.6495628163829
20s(Twenties),650.4038944723618
40s(Fourties),649.0653170359053
60s(Sixties),647.9866666666667
50s(Fifties),646.355581127733


In [0]:
df_plot = df_agegrp_avg_cred.sort("AgeGroup")
x_axis_clm = df_plot.select('AgeGroup').rdd.flatMap(lambda x:x).collect()
y_axis_clm = df_plot.select('Avg_Cred_Score').rdd.flatMap(lambda x:x).collect()


fig = go.Figure()
fig = make_subplots(specs=[[{'secondary_y':True}]])
fig.add_trace(
    go.Scatter(
        x = [x for x in x_axis_clm],
        y = [y for y in y_axis_clm],
        name='AgeGroup',
        text = [x for x in y_axis_clm],
        
    ),secondary_y=False,)

fig.update_layout(
    autosize=False,
    width = 1000,
    height = 500,
    title = 'Credit Score og different Age Groups',
    xaxis_title = ' Age Group',
    yaxis_title = 'Average Credit Score'

)
fig.update_yaxes(automargin=True,title_text='Credit Score og different Age Groups',secondary_y=True)
fig.show()


Average Balance in the account of the Customers under each Age Group

In [0]:
df_agegrp_avg_bal = grpby_avg(df_bank,"Balance","AgeGroup",alias="Avg_Balance") 
display(df_agegrp_avg_bal)

AgeGroup,Avg_Balance
90s(Nineties),123794.77734375
50s(Fifties),83632.9424131545
40s(Fourties),78479.24076312876
60s(Sixties),78058.30283333334
30s(Thirties),75071.79675792935
10s(Teens),74393.86208545919
20s(Twenties),73677.32286309477
70s(Seventies),70416.41878733915
80s(Eighties),32730.88581730769


Average Tenure Period of Customers under each Age Group

In [0]:
df_agegrp_avg_tenure = grpby_avg(df_bank,"Tenure","AgeGroup",alias="Avg_Tenure")
display(df_agegrp_avg_tenure)

AgeGroup,Avg_Tenure
80s(Eighties),5.923076923076923
20s(Twenties),5.105527638190955
40s(Fourties),5.014896867838044
30s(Thirties),4.993787390704096
70s(Seventies),4.985294117647059
50s(Fifties),4.973532796317606
10s(Teens),4.959183673469388
60s(Sixties),4.917333333333334
90s(Nineties),2.0


Finding out what AgeGroup of Customers that are Exited

In [0]:

df_agegrp_exit = grpby_sum(df_bank,"Exited","AgeGroup",alias="Sum_Exited")
display(df_agegrp_exit)

AgeGroup,Sum_Exited
40s(Fourties),806
50s(Fifties),487
30s(Thirties),473
60s(Sixties),132
20s(Twenties),121
70s(Seventies),14
10s(Teens),3
80s(Eighties),1
90s(Nineties),0


Sum of Customers owning Credit Card under AgeGroups

In [0]:
df_agegrp_card = grpby_sum(df_bank,"HasCrCard","AgeGroup",alias="Sum_HasCrCard")
display(df_agegrp_card)

AgeGroup,Sum_HasCrCard
30s(Thirties),3092
40s(Fourties),1830
20s(Twenties),1136
50s(Fifties),588
60s(Sixties),263
70s(Seventies),100
10s(Teens),36
80s(Eighties),9
90s(Nineties),1


Checking how many Customers are active based upon their AgeGroups

In [0]:
df_agegrp_active = grpby_sum(df_bank,"IsActiveMember","AgeGroup",alias="Sum_ActiveMember")
display(df_agegrp_active)

AgeGroup,Sum_ActiveMember
30s(Thirties),2181
40s(Fourties),1222
20s(Twenties),809
50s(Fifties),496
60s(Sixties),281
70s(Seventies),120
10s(Teens),28
80s(Eighties),12
90s(Nineties),2


Customers with Max Tenure, Max CreditScore, and does not have a Card

In [0]:
df_valuable_cust_nocard = df_bank.select("*").filter((df_bank.Tenure==(df_bank.select(max(df_bank.Tenure))).collect()[0][0]) & (df_bank.CreditScore==(df_bank.select(max(df_bank.CreditScore))).collect()[0][0]) & (df_bank.HasCrCard==0) & ((df_bank.Exited==1)))

display(df_valuable_cust_nocard)

RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,AgeGroup
3110,15735878,Law,850,Germany,Female,47,10,134381.52,1,0,0,26812.89,1,40s(Fourties)


Customers with Max Tenure, Max CreditScore, and already has a Card

In [0]:
df_valuable_cust_has_card = df_bank.select("*").filter((df_bank.Tenure==(df_bank.select(max(df_bank.Tenure))).collect()[0][0]) & (df_bank.CreditScore==(df_bank.select(max(df_bank.CreditScore))).collect()[0][0]) & (df_bank.HasCrCard==1) & ((df_bank.Exited==0)))

display(df_valuable_cust_has_card)

RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,AgeGroup
1557,15772777,Onyemachukwu,850,Spain,Female,29,10,0.0,2,1,1,94815.04,0,20s(Twenties)
1800,15612776,McKay,850,Spain,Female,39,10,0.0,2,1,1,143030.1,0,30s(Thirties)
1873,15706593,Ellis,850,Spain,Female,50,10,0.0,2,1,1,33741.84,0,50s(Fifties)
3831,15720713,Chibueze,850,France,Female,29,10,0.0,2,1,1,199775.67,0,20s(Twenties)
5164,15633870,Ozioma,850,France,Female,36,10,0.0,2,1,1,100750.03,0,30s(Thirties)
7372,15624850,Grant,850,France,Male,30,10,153972.89,2,1,0,62811.03,0,30s(Thirties)


Average of Age, Balance, Estimated Salary

In [0]:
df_all_avg = df_bank.select(avg(df_bank.Age),avg(df_bank.Balance),avg(df_bank.Tenure),avg(df_bank.EstimatedSalary))
display(df_all_avg)


avg(Age),avg(Balance),avg(Tenure),avg(EstimatedSalary)
38.9218,76485.88928129883,5.0128,100090.23985771237


Customers that have more than average Credit Score, Tenure, Balance and dosen't have a Credit Card

In [0]:
df = df_bank.select("*").filter((df_bank.Tenure>(df_bank.select(avg(df_bank.Tenure))).collect()[0][0]) & (df_bank.CreditScore>(df_bank.select(avg(df_bank.CreditScore))).collect()[0][0]) & (df_bank.Balance>(df_bank.select(avg(df_bank.Balance))).collect()[0][0]) & (df_bank.HasCrCard==0))
display(df)


RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,AgeGroup
38,15729599,Lorenzo,804,Spain,Male,33,7,76548.6,1,0,1,98453.45,0,30s(Thirties)
54,15702298,Parkhill,655,Germany,Male,41,8,125561.97,1,0,0,164040.94,1,40s(Fourties)
62,15773469,Clark,687,Germany,Female,27,9,152328.88,2,0,0,126494.82,0,20s(Twenties)
97,15738721,Graham,773,Spain,Male,41,9,102827.44,1,0,1,64595.25,0,40s(Fourties)
114,15591100,Chiemela,675,Spain,Male,36,9,106190.55,1,0,1,22994.32,0,30s(Thirties)
115,15609618,Fanucci,721,Germany,Male,28,9,154475.55,2,0,1,101300.94,1,20s(Twenties)
121,15682472,Culbreth,828,France,Male,34,8,129433.34,2,0,0,38131.77,0,30s(Thirties)
122,15580203,Kennedy,674,Spain,Male,39,6,120193.42,1,0,0,100130.95,0,30s(Thirties)
161,15692132,Wilkinson,717,Spain,Female,22,6,101060.25,1,0,1,84699.56,0,20s(Twenties)
163,15630910,Treacy,800,France,Female,49,7,108007.36,1,0,0,47125.11,0,40s(Fourties)


In [0]:
""""
Insights from Analysis:

1. Gender and Female distribution is almost the same
2. Majority of the customers are from France.
3. 20% of people have exited the credit card plans
4. Almost 50% of the customers are not active members in the plan.
5. Germany has no customers with zero balance in their accounts.
6. Highest Credit Score is 850 - Chiemezie and many others have it.
7. Age groups 30s, 40s, and 50s have the lowest average credit score.
8. Age groups 90s and 50s have the highest average balance in their account.
9. Suprisingly the Age group 20s has the 2nd longest tenure period (avg).
10. There are 408 customers that have more than average Credit Score, Tenure, Balance and still dosen't have a Credit Card
11.There is one customer 15735878 with Max Tenure, Max CreditScore, and does not have a Card


Manikandan Mohan 
M1074315

""""
    